In [14]:
import torch
import random
import numpy as np
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import utils as U
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "textattack/bert-base-uncased-imdb"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/511 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [27]:
from datasets import load_dataset
imdb = load_dataset("imdb")
test_texts = imdb["test"]["text"]
test_labels = imdb["test"]["label"]


In [30]:
import random, json

def apply_mutations(texts, mutation_dict, mutation_type="typo", word_mutation_prob=0.1):
    mutated_texts = []
    for text in texts:
        words = text.split()
        new_words = []
        for w in words:
            key = w.lower()
            if key in mutation_dict and mutation_dict[key] and random.random() < word_mutation_prob:
                replacement = random.choice(mutation_dict[key])
                new_words.append(replacement)
            else:
                new_words.append(w)
        mutated_texts.append(" ".join(new_words))
    return mutated_texts


In [33]:
from torch.utils.data import DataLoader
from transformers import TextClassificationPipeline

def evaluate_accuracy(model, tokenizer, texts, labels, batch_size=32):
    pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=False, truncation=True, device=-1)
    preds = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        outputs = pipe(batch)
        preds += [int(output['label'] == 'LABEL_1') for output in outputs]
    correct = sum([p == l for p, l in zip(preds, labels)])
    return 100 * correct / len(labels)


In [34]:
with open("mutation_data/misspellings.json") as f:
    typo_dict = json.load(f)

with open("mutation_data/synonyms.json") as f:
    synonym_dict = json.load(f)

mutation_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]

for prob in mutation_levels:
    typo_texts = apply_mutations(test_texts, typo_dict, "typo", word_mutation_prob=prob)
    acc = evaluate_accuracy(model, tokenizer, typo_texts, test_labels)
    print(f"Typo mutation at {int(prob*100)}%: Accuracy = {acc:.2f}%")


Device set to use cpu
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


KeyboardInterrupt: 